### NASA Mars News

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragragh Text. Assign the text to variables that you can reference later.

```python
# Example:
news_title = "NASA's Next Mars Mission to Investigate Interior of Red Planet"

news_p = "Preparation of NASA's next spacecraft to Mars, InSight, has ramped up this summer, on course for launch next May from Vandenberg Air Force Base in central California -- the first interplanetary launch in history from America's West Coast."
```

In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
# have to run browser object since 
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# URL of NASA's mars news
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# launch browser
browser.visit(url)

# create beautifulsoup object
soup = BeautifulSoup(browser.html, 'html.parser')

In [16]:
# find latest news date, title and body (first appearanse of a tag in html)
news_date = soup.find('div', class_='list_date').text
news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='article_teaser_body').text

In [17]:
# print latest news title and paragraph
print(f'Date : {news_date}.\nTitle: {news_title}.\nText : {news_p}')

Date : May 23, 2018.
Title: InSight Steers Toward Mars.
Text : The spacecraft has completed its first trajectory correction maneuver.


### JPL Mars Space Images - Featured Image

* Visit the url for JPL's Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).

* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.

* Make sure to find the image url to the full size `.jpg` image.

* Make sure to save a complete url string for this image.

In [22]:
# URL of mars images page
url = 'https://www.jpl.nasa.gov/'
url1 = 'spaceimages/?search=&category=Mars'

# launch browser
browser.visit(url+url1)

# create beautifulsoup object
soup = BeautifulSoup(browser.html, 'html.parser')

In [29]:
# remove last '/' symbol from url
# locate the 'style' attribute of 'article' tag
# split by quotes and access an element of resulting list
# combine the two text elements get the image url
featured_image_url = url[:-1] + soup.find('article', class_='carousel_item')['style'].split("'")[1]
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17551-1920x1200.jpg'

### Mars Weather

* Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.

In [37]:
# twitter url to visit
url = 'https://twitter.com/marswxreport?lang=en'

# launch browser
browser.visit(url+url1)

# create beautifulsoup object
soup = BeautifulSoup(browser.html, 'html.parser')

In [43]:
# localate the first tweet and extract text from it
mars_weather = soup.find('div', class_='js-tweet-text-container').text.split('\n')[1]
mars_weather

'Sol 2062 (May 25, 2018), Sunny, high 2C/35F, low -72C/-97F, pressure at 7.45 hPa, daylight 05:19-17:20'